In [1]:
# data saving imports
import pickle
import os

# time tracking
from datetime import datetime as dt

# multiprocessing
from multiprocessing import Pool

import statsmodels.formula.api as smf
import statsmodels.tools.sm_exceptions as smExceptions

# custom imports
from regression_class import RedditRegression as RR
from regression_class import TimestampClass as tc

import warnings
from numpy.linalg import LinAlgError

In [2]:
TEST_INFILE = "test_data_4_days.p"
test_data = pickle.load(open(TEST_INFILE, 'rb'))
regression_df = test_data["regression_data"]
thread_df = test_data["all_data"]

In [3]:
X_COLS = [
    "sentiment_sign",
    "sentiment_magnitude",
    "time_in_secs",
    "num_dayofweek",
    "activity_ratio",
    "mean_author_sentiment_sign",
    "mean_author_sentiment_magnitude",
    "log_author_all_activity_count",
]
quantiles = [0.25, 0.5, 0.75]
thresholds2 = {
    "author_all_activity_count": 2,
    "thread_size": 2,
}
thresholds1 = {
    "author_all_activity_count": 2,
}

regression_params = {
    'regression_data': regression_df['books'],
    'thread_data': thread_df['books'],
    'collection_window': 1,
    'model_window': 2,
    'validation_window': 1,
    'FSS': True,
    'x_cols': X_COLS,
    'scale': True,
}

In [4]:
regression_types = ['logistic', 'linear', 'mnlogit']
to_vary = {
    'name': [f'books_{x}' for x in regression_types],
    'regression_type': regression_types,
    'y_col': ['success', 'thread_size', 'thread_size'],
    'metrics': [['auc'], ['r2'], ['mnlogit_accuracy', 'mnlogit_aucs', "mnlogit_mean_auc"]],
    'thresholds': [thresholds1, thresholds2, thresholds2],
    'quantiles': [[], [], quantiles],
}

regressions = {}
for i, regression_type in enumerate(regression_types):
    in_params = regression_params.copy()
    for key in to_vary:
        in_params[key] = to_vary[key][i]
    regressions[regression_type] = RR(in_params)

In [5]:
testreg = regressions['logistic']

In [6]:
testreg.main()

/home/cara/Documents/reddit_analyses/reddit_analyses/regression_class.py:905: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data.drop(labels=to_drop, axis=1, inplace=True)
/home/cara/Documents/reddit_analyses/reddit_analyses/regression_class.py:905: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data.drop(labels=to_drop, axis=1, inplace=True)
/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/nu

Running FSS
Model 1
Optimization terminated successfully.
         Current function value: 0.585670
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.585670
         Iterations 5
Model 2
         Current function value: 0.440824
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.440824
         Iterations: 27
         Function evaluations: 28
         Gradient evaluations: 28
Optimization terminated successfully.
         Current function value: 0.440824
         Iterations: 27
         Function evaluations: 28
         Gradient evaluations: 28
Model 3
         Current function value: 0.401594
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.401594
         Iterations: 32
         Function evaluations: 33
         Gradient evaluations: 33
Optimization terminated successfully.
         Current function value: 0.401594
         Iterations: 32
        

In [7]:
testreg.get_cal_val_data()
testreg.sm_modstrings = testreg.run_FSS()

/home/cara/Documents/reddit_analyses/reddit_analyses/regression_class.py:905: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data.drop(labels=to_drop, axis=1, inplace=True)
/home/cara/Documents/reddit_analyses/reddit_analyses/regression_class.py:905: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data.drop(labels=to_drop, axis=1, inplace=True)
/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/nu

In [8]:
model_results = {}
param_dict = {}
testreg.smf_models = {}

In [9]:
smf_model = (
            getattr(smf, testreg.SMF_FUNCTIONS[testreg.regression_params["regression_type"]])(
                testreg.sm_modstrings[8], data=testreg.__model_data__["cal"]
            )
        )

In [9]:
smf_model = testreg.run_regression(5)

         Current function value: 0.212076
         Iterations: 35
         Current function value: 0.212076
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
Optimization terminated successfully.
         Current function value: 0.212076
         Iterations: 37
         Function evaluations: 38
         Gradient evaluations: 38
         Current function value: 0.212223
         Iterations: 100
         Function evaluations: 316
         Gradient evaluations: 316
         Current function value: 0.212113
         Iterations: 150
         Function evaluations: 483
         Gradient evaluations: 483
Optimization terminated successfully.
         Current function value: 0.212076
         Iterations: 167
         Function evaluations: 533
         Gradient evaluations: 533
Optimization terminated successfully.
         Current function value: 0.212076
         Iterations: 167
         Function evaluations: 533
         Gradient evaluations: 533


In [10]:
smf_model.cov_params()

,Intercept,sentiment_sign,sentiment_magnitude,time_in_secs,num_dayofweek,mean_author_sentiment_magnitude
Intercept,1.688048e+15,-9.089406e+14,4.136735e+15,1.364283,5.024110e+07,5.155681e+07
sentiment_sign,-9.089406e+14,4.894252e+14,-2.227453e+15,0.738022,3.329976e+07,3.424405e+07
sentiment_magnitude,4.136735e+15,-2.227453e+15,1.013750e+16,3.923707,1.434095e+08,1.470369e+08
time_in_secs,1.364283e+00,7.380225e-01,3.923707e+00,0.781838,3.035245e+00,2.537926e+00
num_dayofweek,5.024110e+07,3.329976e+07,1.434095e+08,3.035245,9.534548e+07,9.793685e+07
mean_author_sentiment_magnitude,5.155681e+07,3.424405e+07,1.470369e+08,2.537926,9.793685e+07,1.006756e+08


In [11]:
testreg.get_model_metrics_from_smf_mod(smf_model)

,param,stderr,pvalue
Intercept,-6.166670,4.108586e+07,1.000000
sentiment_sign,58.070446,2.212296e+07,0.999998
sentiment_magnitude,14.111301,1.006851e+08,1.000000
time_in_secs,-0.036586,8.842163e-01,0.966995
num_dayofweek,77.337833,9.764501e+03,0.993681
mean_author_sentiment_magnitude,73.723462,1.003373e+04,0.994138


In [10]:
smf_model = testreg.fit_smf_model(smf_model)

         Current function value: inf
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.212076
         Iterations: 32
         Function evaluations: 33
         Gradient evaluations: 33
         Current function value: 0.212093
         Iterations: 35
         Function evaluations: 98
         Gradient evaluations: 98
         Current function value: 0.212093
         Iterations: 35
         Function evaluations: 98
         Gradient evaluations: 98
         Current function value: 0.212093
         Iterations: 35
         Function evaluations: 98
         Gradient evaluations: 98
         Current function value: 0.212093
         Iterations: 35
         Function evaluations: 98
         Gradient evaluations: 98
         Current function value: 0.212093
         Iterations: 35
         Function evaluations: 98
         Gradient evaluations: 98
         Current function value: 0.212093
         Iterations: 35
         Function evaluations: 

/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [11]:
model_results = {}
model_results["model"] = testreg.sm_modstrings[5]


In [16]:
run_again = True
run_counter = 0
max_runs = 10
kwargs_dict = {}

while run_again==True and run_counter<max_runs:
    run_again, kwargs_dict = testreg.manage_convergence_warnings(smf_model.fit, **kwargs_dict)
    print(run_again, kwargs_dict)
    run_counter += 1

         Current function value: 0.212076
         Iterations: 35
True {'method': 'bfgs'}
         Current function value: 0.212076
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
True {'method': 'bfgs', 'maxiter': 100}
Optimization terminated successfully.
         Current function value: 0.212076
         Iterations: 37
         Function evaluations: 38
         Gradient evaluations: 38


TypeError: cannot unpack non-iterable NoneType object

In [11]:
smf_model = smf.logit(testreg.sm_modstrings[8], data = testreg.__model_data__['cal'])

In [24]:
testreg.manage_convergence_warnings(smf_model.fit, method='cg')

         Current function value: 0.212093
         Iterations: 35
         Function evaluations: 98
         Gradient evaluations: 98


(True, {'method': 'cg'})

In [26]:
smf_model.fit(method='bfgs', maxiter=100)

Optimization terminated successfully.
         Current function value: 0.212076
         Iterations: 32
         Function evaluations: 33
         Gradient evaluations: 33


/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


In [21]:
reg_params = smf_model.fit_regularized().params

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.2120758210671058
            Iterations: 48
            Function evaluations: 49
            Gradient evaluations: 48


In [ ]:
 start_params=reg_params

In [107]:
with warnings.catch_warnings(record=True) as w:
    smf_model.fit(maxiter=55)

         Current function value: 0.212076
         Iterations: 55


In [160]:
smf_model.fit(maxiter=55)

         Current function value: 0.212076
         Iterations: 55


/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [117]:
maxiter = 35

In [157]:
def model_to_run(fit_function, **kwargs):
    run_again=False
    with warnings.catch_warnings(record=True) as w:
        try:
            fit_function(**kwargs)
        except LinAlgError:
            kwargs['method'] = 'bfgs'
            run_again=True
            return run_again, kwargs
    if len(w)>0:
        for w_i in w:
            if w_i.category == RuntimeWarning:
                kwargs['method'] = 'bfgs'
                return run_again, kwargs
            elif w_i.category == smExceptions.ConvergenceWarning:
                if 'maxiter' in kwargs:
                    kwargs['maxiter']*=2
                else:
                    kwargs['maxiter'] = 100
                return run_again, kwargs

    
    

In [158]:
run_again = True
kwargs_dict = {}
while run_again:
    run_again, kwargs_dict = model_to_run(smf_model.fit, **kwargs_dict)

smf_model.fit(**kwargs_dict)

         Current function value: 0.212076
         Iterations: 35
         Current function value: 0.212076
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
Optimization terminated successfully.
         Current function value: 0.212076
         Iterations: 37
         Function evaluations: 38
         Gradient evaluations: 38


/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


In [155]:
smf_model.fit(maxiter=100, method='bfgs')

Optimization terminated successfully.
         Current function value: 0.212076
         Iterations: 37
         Function evaluations: 38
         Gradient evaluations: 38


/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


In [154]:
kwargs_dict

{'maxiter': 100, 'method': 'bfgs'}

In [133]:
model_to_run(smf_model.fit, maxiter=60, method='bfgs')

Optimization terminated successfully.
         Current function value: 0.212076
         Iterations: 37
         Function evaluations: 38
         Gradient evaluations: 38


In [113]:
for w_i in w:
    print(w_i.category)
    print(w_i.category == RuntimeWarning)
    print(w_i.category == smExceptions.ConvergenceWarning)

<class 'RuntimeWarning'>
True
False
<class 'statsmodels.tools.sm_exceptions.ConvergenceWarning'>
False
True


In [98]:

try:
    modfit = smf_model.fit()
except LinAlgError:
    modfit = smf_model.fit(maxiter=55)
except RuntimeWarning:
    modfit = smf_model

         Current function value: 0.212076
         Iterations: 35
         Current function value: 0.212076
         Iterations: 55


/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [101]:
smf_model.fit(method='bfgs', maxiter=55)

Optimization terminated successfully.
         Current function value: 0.212076
         Iterations: 37
         Function evaluations: 38
         Gradient evaluations: 38


/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


In [71]:
mod_key = 5
testreg.smf_models[mod_key], model_results[mod_key] = testreg.run_regression(
                mod_key
            )

         Current function value: 0.212076
         Iterations: 35


/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


LinAlgError: Singular matrix

In [20]:
testreg.run_models()

Model 1
Optimization terminated successfully.
         Current function value: 0.585670
         Iterations 5
Model 2
         Current function value: 0.440824
         Iterations: 35
Model 3
         Current function value: 0.407509
         Iterations: 35
Model 4
         Current function value: 0.407507
         Iterations: 35
Model 5
         Current function value: inf
         Iterations: 35


/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/cara/anaconda3/envs/redditenv/lib/python3.10/site-packages/statsmodels/di

LinAlgError: Singular matrix

In [37]:
testreg.calc_author_thread_counts()
testreg.get_regression_model_data()
testreg.get_regression_model_data(calval='val')

In [39]:
testreg.perform_scaling()

In [23]:
# start running
for key in regressions:
    print(f"Running {key}")
    regressions[key].main()

Running logistic


TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[int64]'>)

In [25]:
regressions['logistic']

In [3]:
# outfiles
OUTDIR = "regression_test_outputs"
METRICS_OUTFILE = "regression_metrics"

# subreddits to look at
subreddits = ["books", "crypto", "conspiracy", "politics"]

# regression types to run
regression_types = ["logistic", "linear", "mnlogit"]

# get outdir names for all regression types
out_subdirs = {}
for regtype in regression_types:
    out_subdirs[regtype] = f"{OUTDIR}/{regtype}"

# make out params dict to save spreadsheets
out_params_dict = {}

In [8]:
def run_regression_type(regression_type):
    print(dt.now())
    print(f"\n    ## Regression type: {regression_type}##")
    input_params = regression_params[regression_type].copy()

    # place to store logregs
    subreddit_logregs = {}
    try:
        for subreddit in subreddits:
            print(f"#{subreddit}#")
            input_params["name"] = subreddit
            input_params["regression_data"] = regression_df[subreddit]
            input_params["thread_data"] = thread_df[subreddit]

            subreddit_logregs[subreddit] = RR(regression_params=input_params)

            subreddit_logregs[subreddit].main()
    except TypeError:
        return subreddit_logregs

    # dump pickle results
    outstring = f"{out_subdirs[regression_type]}/{METRICS_OUTFILE}.p"
    print(f"\n\n\n   DUMPING RESULTS TO \n{outstring}\n\n\n")
    pickle.dump(
        subreddit_logregs, open(outstring, "wb",),
    )

In [5]:
# make out directories
for outdirname in ([OUTDIR] + list(out_subdirs.values())):
    if not os.path.isdir(outdirname):
        os.mkdir(outdirname)

# read in files
#print(dt.now())
#print("reading in input files")
# TESTING
#regression_df = pickle.load(open(REGRESSION_INFILE, "rb"))
#thread_df = pickle.load(open(THREAD_INFILE, "rb"))

print("Creating parameter dictionaries")
# fixed regression params
X_COLS = [
    "sentiment_sign",
    "sentiment_magnitude",
    "hour",
    "time_in_secs",
    "num_dayofweek",
    "activity_ratio",
    "mean_author_sentiment_sign",
    "mean_author_sentiment_magnitude",
    "author_all_activity_count",
]

fixed_regression_params = {
    "collection_window": 7,
    "model_window": 14,
    "validation_window": 7,
    "FSS": True,
    "x_cols": X_COLS,
    "scale": True,
}

# variable regression params
quantiles = [0.25, 0.5, 0.75]
thresholds2 = {
    "author_all_activity_count": 2,
    "thread_size": 2,
}
thresholds1 = {
    "author_all_activity_count": 2,
}

to_vary = {
    "regression_type": regression_types,
    "y_col": ["success", "thread_size", "thread_size"],
    "metrics": [
        ["auc"],
        ["r2"],
        ["mnlogit_accuracy", "mnlogit_aucs", "mnlogit_mean_auc"],
    ],
    "thresholds": [thresholds1, thresholds2, thresholds2],
    "quantiles": [[], [], quantiles],
}

regression_params = {}
for i, regtype in enumerate(regression_types):
    regression_params[regtype] = fixed_regression_params.copy()
    for key in to_vary:
        regression_params[regtype][key] = to_vary[key][i]

Creating parameter dictionaries


In [13]:
subreddit_logregs = {}
for key in regression_types:
    subreddit_logregs[key] = run_regression_type(key)

2024-03-11 14:00:19.658496

    ## Regression type: logistic##
#books#
2024-03-11 14:00:20.910123

    ## Regression type: linear##
#books#
2024-03-11 14:00:22.071844

    ## Regression type: mnlogit##
#books#


In [18]:
model_data = subreddit_logregs['logistic']['books'].model_data

In [20]:
model_data

{'cal': Empty DataFrame
 Columns: [thread_id, thread_size, authors, timestamp, author, score, subject_sentiment_score, sentiment_sign, sentiment_magnitude, success, author_all_activity_count, activity_ratio, mean_author_sentiment_sign, mean_author_sentiment_magnitude, time_in_secs, num_dayofweek, hour]
 Index: [],
 'val': Empty DataFrame
 Columns: [thread_id, thread_size, authors, timestamp, author, score, subject_sentiment_score, sentiment_sign, sentiment_magnitude, success, author_all_activity_count, activity_ratio, mean_author_sentiment_sign, mean_author_sentiment_magnitude, time_in_secs, num_dayofweek, hour]
 Index: []}

In [ ]:
x_data = self.model_data[calval][self.regression_params["x_cols"]]